<a href="https://colab.research.google.com/github/agnieszkaborowiecka/dw_matrix_car/blob/master/day3_simple_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 3.5MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 3.5MB/s 


In [4]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [5]:
cd "/content/drive/My Drive/Colab Notebooks/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix_two/dw_matrix_car


In [6]:
ls

car.h5  data/  day2_visualisation.ipynb  dw_matrix_car/  LICENSE  README.md


Wczytywanie danych


In [7]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [8]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

Dummy Model

In [9]:
df.select_dtypes(np. number).columns

Index(['price_value', 'car_id'], dtype='object')

In [11]:
feats = ['car_id']
x = df[ feats ].values
y = df[ ['price_value'] ].values

model = DummyRegressor()
model.fit(x, y)
y_pred = model.predict(x)

mae(y, y_pred)

39465.934630440985

In [12]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [14]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [15]:
df['price_currency'].value_counts(normalize=True) * 100

PLN    99.80844
EUR     0.19156
Name: price_currency, dtype: float64

Usuwamy wszystkie ogłoszenia z EUR

In [16]:
df = df[ df['price_currency'] != 'EUR' ]
df.shape

(106290, 155)

##Features (przygotowanie cech)


In [17]:
df.head

<bound method NDFrame.head of                                                breadcrumb  ...  car_id
0         [Osobowe, Dacia, Logan, Dacia Logan II (2012-)]  ...       0
2       [Osobowe, Peugeot, 3008, Peugeot 3008 I (2009-...  ...       2
3       [Osobowe, Porsche, Cayenne, Porsche Cayenne II...  ...       3
4       [Osobowe, Peugeot, 308, Peugeot 308 T7 (2008-2...  ...       4
5       [Osobowe, Volkswagen, Golf, Volkswagen Golf VI...  ...       5
...                                                   ...  ...     ...
160609  [Osobowe, Hyundai, i20, Hyundai i20 I (2008-20...  ...  160609
160610                            [Osobowe, Škoda, RAPID]  ...  160610
160611  [Osobowe, Opel, Zafira, Opel Zafira B (2005-20...  ...  160611
160614  [Osobowe, Volkswagen, Passat, Volkswagen Passa...  ...  160614
160615                            [Osobowe, Peugeot, 207]  ...  160615

[106290 rows x 155 columns]>

In [18]:
for feat in df.columns:

  print(feat)

breadcrumb
created_at
price_currency
price_details
price_value
seller_address
seller_name
seller_type
feature_czujniki-parkowania-przednie
feature_poduszka-powietrzna-chroniąca-kolana
feature_kurtyny-powietrzne
feature_klimatyzacja-dwustrefowa
feature_światła-led
feature_czujnik-zmierzchu
feature_elektrycznie-ustawiane-lusterka
feature_asr-(kontrola-trakcji)
feature_poduszka-powietrzna-kierowcy
feature_cd
feature_elektryczne-szyby-przednie
feature_poduszka-powietrzna-pasażera
feature_system-start-stop
feature_światła-do-jazdy-dziennej
feature_komputer-pokładowy
feature_elektryczne-szyby-tylne
feature_klimatyzacja-manualna
feature_tapicerka-welurowa
feature_czujnik-deszczu
feature_światła-przeciwmgielne
feature_ogrzewanie-postojowe
feature_radio-niefabryczne
feature_regulowane-zawieszenie
feature_ogranicznik-prędkości
feature_zmieniarka-cd
feature_szyberdach
feature_isofix
feature_asystent-pasa-ruchu
feature_alufelgi
feature_bluetooth
feature_nawigacja-gps
feature_asystent-parkowania
fe

In [19]:
df['param_color'].factorize()

(array([-1, -1, -1, ..., -1, -1, -1]),
 Index(['Czarny', 'Srebrny', 'Złoty', 'Bordowy', 'Biały', 'Szary', 'Beżowy',
        'Zielony', 'Fioletowy', 'Niebieski', 'Inny kolor', 'Brązowy', 'Żółty'],
       dtype='object'))

In [24]:
df['param_color'].factorize()[0]

array([-1, -1, -1, ..., -1, -1, -1])

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [31]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x ]
cat_feats = [x for x in cat_feats if 'price' not in x ]
len(cat_feats)

151

In [33]:
x = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, x, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19566.588937368324

In [35]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(x, y)

imp = PermutationImportance(m, random_state=0).fit(x, y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2533 ± 0.0040,param_napęd__cat
0.2008 ± 0.0047,param_faktura-vat__cat
0.1943 ± 0.0088,param_stan__cat
0.1423 ± 0.0086,param_rok-produkcji__cat
0.0629 ± 0.0047,param_moc__cat
0.0424 ± 0.0014,feature_kamera-cofania__cat
0.0412 ± 0.0008,param_skrzynia-biegów__cat
0.0286 ± 0.0037,param_marka-pojazdu__cat
0.0191 ± 0.0022,param_pojemność-skokowa__cat
0.0163 ± 0.0005,feature_bluetooth__cat


In [36]:
!pwd

/content/drive/My Drive/Colab Notebooks/matrix_two/dw_matrix_car


wysłanie na githuba (nie działa!!!)

In [0]:
##GITHUB_TOKEN = 'GITHUB_TOKEN'
##GITHUB_URL = 'https://{}@github.com/agnieszkaborowiecka/dw_matrix_car.git'.format(GITHUB_TOKEN)

In [0]:
##!git config --global user.email "agnieszka.borowiecka@gmail.com"
##!git config --global user.name "Agnieszka"

In [0]:
##!git add 'day3_simple_models.ipynb'
##!git commit -m "add models"
##!git push -u origin master